# Web-scraping the data through aplha vantage api for multiple forex currencies for the last ten years 

In [10]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Replace with your Alpha Vantage API key
api_key = '89PMD611NPUTS89W'

# Function to fetch forex data
def fetch_forex_data(from_currency, to_currency):
    url = f'https://www.alphavantage.co/query'
    params = {
        'function': 'FX_DAILY',
        'from_symbol': from_currency,
        'to_symbol': to_currency,
        'apikey': api_key,
        'outputsize': 'full'
    }
    response = requests.get(url, params=params)
    data = response.json()
    
    # Extracting the time series data
    time_series = data.get('Time Series FX (Daily)', {})
    
    # Converting to DataFrame
    df = pd.DataFrame.from_dict(time_series, orient='index')
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()
    
    # Adding a column for the currency pair
    df['Currency Pair'] = f'{from_currency}/{to_currency}'
    
    # Filtering data for the last 10 years
    ten_years_ago = datetime.now() - timedelta(days=365*20)
    df = df[df.index >= ten_years_ago]
    
    return df

# List of common currency pairs to fetch which are allowed to trade in india 
currency_pairs = [
    ('USD', 'INR'), ('EUR', 'USD'), ('GBP', 'USD'), ('USD', 'JPY'), ('EUR', 'INR'),
    ('JPY', 'INR'), ('GBP', 'INR')
]

# Fetching data for all currency pairs and combining into a single DataFrame
all_data = pd.DataFrame()
for from_currency, to_currency in currency_pairs:
    forex_data = fetch_forex_data(from_currency, to_currency)
    all_data = pd.concat([all_data, forex_data])

# Resetting index for the combined DataFrame
all_data.reset_index(inplace=True)
all_data.rename(columns={'index': 'Date',"1. open":"Open_price","2. high":"Day_high","3. low":"Day_low","4. close":"Closing_price"}, inplace=True)

print(all_data)


            Date Open_price   Day_high    Day_low Closing_price Currency Pair
0     2014-11-07   61.39000   61.62000   61.34000      61.40000       USD/INR
1     2014-11-10   61.50000   61.63500   61.34000      61.49500       USD/INR
2     2014-11-11   61.53000   61.55500   61.50500      61.50800       USD/INR
3     2014-11-12   61.50800   61.56000   61.35000      61.39100       USD/INR
4     2014-11-13   61.36800   61.62300   61.35000      61.56500       USD/INR
...          ...        ...        ...        ...           ...           ...
18223 2024-10-23  109.13110  109.24560  108.57490     108.58750       GBP/INR
18224 2024-10-24  108.53600  109.09080  108.47760     108.97390       GBP/INR
18225 2024-10-25  109.05690  109.20100  108.87720     108.96040       GBP/INR
18226 2024-10-28  108.93470  109.20340  108.74210     108.99700       GBP/INR
18227 2024-10-29  109.02870  109.04350  108.90390     108.98320       GBP/INR

[18228 rows x 6 columns]


In [11]:
all_data.head()

,Date,Open_price,Day_high,Day_low,Closing_price,Currency Pair
0,2014-11-07,61.39000,61.62000,61.34000,61.40000,USD/INR
1,2014-11-10,61.50000,61.63500,61.34000,61.49500,USD/INR
2,2014-11-11,61.53000,61.55500,61.50500,61.50800,USD/INR
3,2014-11-12,61.50800,61.56000,61.35000,61.39100,USD/INR
4,2014-11-13,61.36800,61.62300,61.35000,61.56500,USD/INR


In [12]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18228 entries, 0 to 18227
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           18228 non-null  datetime64[ns]
 1   Open_price     18228 non-null  object        
 2   Day_high       18228 non-null  object        
 3   Day_low        18228 non-null  object        
 4   Closing_price  18228 non-null  object        
 5   Currency Pair  18228 non-null  object        
dtypes: datetime64[ns](1), object(5)
memory usage: 854.6+ KB


#### According to the above stats we come to know that the columns are object dtype hence we need to typecast them into float values 

In [13]:
all_data["Open_price"] = all_data["Open_price"].astype(float)
all_data["Day_high"] = all_data["Day_high"].astype(float)
all_data["Day_low"] = all_data["Day_low"].astype(float)
all_data["Closing_price"] = all_data["Closing_price"].astype(float)

In [14]:
all_data.dtypes

Date             datetime64[ns]
Open_price              float64
Day_high                float64
Day_low                 float64
Closing_price           float64
Currency Pair            object
dtype: object

In [15]:
all_data.describe()

,Date,Open_price,Day_high,Day_low,Closing_price
count,18228,18228.000000,18228.000000,18228.000000,18228.000000
mean,2019-11-01 06:13:30.533245696,53.091485,53.281500,52.908917,53.093291
min,2014-10-28 00:00:00,0.507380,0.511690,0.506870,0.507550
25%,2017-05-03 00:00:00,1.156370,1.160900,1.152615,1.156540
50%,2019-11-01 00:00:00,70.673900,70.980050,70.412000,70.667950
75%,2022-05-02 00:00:00,91.099575,91.454425,90.732775,91.084875
max,2024-10-29 00:00:00,161.621000,161.942000,161.300000,161.621000
std,NaN,47.637176,47.809879,47.467097,47.638149


## Outlier Detection

In [16]:
import plotly.express as px 
colors = px.colors.qualitative.Plotly  # Color palette
num_colors = len(colors)  # Number of colors available

for idx, i in enumerate(all_data.select_dtypes("number").columns):
    color = colors[idx % num_colors]  # Cycle through colors for each column
    
    for j in all_data['Currency Pair'].unique(): # For each currency pair
        data = all_data.loc[all_data['Currency Pair'] == j]
        
        fig = px.box(data, y=i, title=f"Outlier Detection for column: {i}, Currency Pair: {j}")
        fig.update_traces(marker_color=color)  # Set the box color

        fig.show()

#### This shows that there are outliers in the data when we check according to different currency pairs

## Outlier Handling

In [33]:
def replace_outliers(df, column):
    # Calculate IQR for the column
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    # Define outlier boundaries
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Replace outliers with the median of the column
    median = df[column].median()
    df[column] = df[column].apply(lambda x: median if x < lower_bound or x > upper_bound else x)
    return df

# Replace outliers for each currency pair
for currency in all_data['Currency Pair'].unique():
    # Filter data for the current currency pair
    currency_data = all_data[all_data['Currency Pair'] == currency]

    # Apply outlier replacement for each numeric column in the currency-specific DataFrame
    for column in currency_data.select_dtypes('number').columns:
        currency_data = replace_outliers(currency_data, column)
    
    # Update the original DataFrame with modified currency-specific data
    all_data.update(currency_data)

print("Outliers replaced successfully.")


Outliers replaced successfully.


C:\Users\Sejal Hanmante\AppData\Local\Temp\ipykernel_23076\3776404046.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Sejal Hanmante\AppData\Local\Temp\ipykernel_23076\3776404046.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Sejal Hanmante\AppData\Local\Temp\ipykernel_23076\3776404046.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

# Time-Series Analysis 

In [36]:
all_data["Currency Pair"].unique()

array(['EUR/USD', 'GBP/INR'], dtype=object)

In [34]:
import plotly.express as px
def plot_df(df, x, y, title="", xlabel='Date', ylabel='Open_price'):
    fig = px.line(df, x=x, y=y, title=title, labels={x: xlabel, y: ylabel}, 
                  line_shape='linear', markers=True)
    fig.update_layout(title=title, xaxis_title=xlabel, yaxis_title=ylabel)
    fig.show() 

In [38]:
for i in all_data["Currency Pair"].unique():
    a = all_data[all_data["Currency Pair"]==i][["Date","Open_price","Currency Pair"]]
    plot_df(a, x='Date', y="Open_price", title=f'Open Price for last 10 years for the currency pair : {i}')


c:\ProgramData\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



c:\ProgramData\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



## Trend and Seasonality Plot 

In [35]:
import plotly.graph_objects as go

def plot_df(a, x, y, title="", xlabel='Date', ylabel='Open_price'):
    # Create a Plotly figure
    fig = go.Figure()
    
    # Add a line plot
    fig.add_trace(go.Scatter(x=x, y=y, mode='lines', line=dict(color='blue')))
    
    # Update layout
    fig.update_layout(
        title=title,
        xaxis_title=xlabel,
        yaxis_title=ylabel,
        height=400,
        width=800
    )
    
    # Show the figure
    fig.show()

# Call the function
for i in all_data["Currency Pair"].unique():
    a = all_data[all_data["Currency Pair"]==i][["Date","Open_price","Currency Pair"]]
    plot_df(a, x=a['Date'], y=a['Open_price'], title=f'Trend and Seasonality {i}')


## Check Stationarity of the data 

1. ADFuller Test 